## Library

In [1]:
import json
import pandas as pd
from pathlib import Path
from jsonschema import validate, ValidationError
from datasets import Dataset, load_dataset

/home/fishmon/.conda/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Dataset

In [2]:
dataset = load_dataset("json", data_files="./dataset/netpro_chatml_thought.jsonl")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 33262
    })
})

In [4]:
dataset = dataset["train"]  # Access the 'train' split

In [5]:
print("\nOutput Type:", type(dataset))
print("Dataset Info:")
print(dataset)


Output Type: <class 'datasets.arrow_dataset.Dataset'>
Dataset Info:
Dataset({
    features: ['conversations'],
    num_rows: 33262
})


In [6]:
print(dataset[0])  # Print the first example

{'conversations': [{'content': 'You are an Expert Website Classifier tasked with categorizing websites (using provided Domain & Content) into distinct categories: **0 - Benign**, **1 - Gambling**, **2 - Pornography**, or **3 - Harmful**. Output strict JSON including classification, reason, and a point-based confidence score (0-100). This aids a sophisticated website prediction system for digital safety.\n\n### **Categories & Definitions:**\n\n-   **0 - Benign:** General info, news, entertainment, services, e-commerce, educational sites, blogs, informational pages, and general entertainment sites. No gambling/porn themes.\n    -   **Includes:** Educational discussion of sensitive topics (drugs, gambling, adult, illegal), suggestive content (dating, lingerie) *without* explicit material, drugs for educational or news purposes (e.g., addiction recovery, medical cannabis research).\n\n-   **1 - Gambling:** Promotes/facilitates betting, casino, poker, lottery, wagering. Includes online bett

## Load Model

In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 30000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.5.1 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.10 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3090 Ti. Num GPUs = 1. Max memory: 23.551 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [9]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [10]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [11]:
dataset[5]["conversations"]

[{'content': 'You are an Expert Website Classifier tasked with categorizing websites (using provided Domain & Content) into distinct categories: **0 - Benign**, **1 - Gambling**, **2 - Pornography**, or **3 - Harmful**. Output strict JSON including classification, reason, and a point-based confidence score (0-100). This aids a sophisticated website prediction system for digital safety.\n\n### **Categories & Definitions:**\n\n-   **0 - Benign:** General info, news, entertainment, services, e-commerce, educational sites, blogs, informational pages, and general entertainment sites. No gambling/porn themes.\n    -   **Includes:** Educational discussion of sensitive topics (drugs, gambling, adult, illegal), suggestive content (dating, lingerie) *without* explicit material, drugs for educational or news purposes (e.g., addiction recovery, medical cannabis research).\n\n-   **1 - Gambling:** Promotes/facilitates betting, casino, poker, lottery, wagering. Includes online betting platforms, cas

In [12]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an Expert Website Classifier tasked with categorizing websites (using provided Domain & Content) into distinct categories: **0 - Benign**, **1 - Gambling**, **2 - Pornography**, or **3 - Harmful**. Output strict JSON including classification, reason, and a point-based confidence score (0-100). This aids a sophisticated website prediction system for digital safety.\n\n### **Categories & Definitions:**\n\n-   **0 - Benign:** General info, news, entertainment, services, e-commerce, educational sites, blogs, informational pages, and general entertainment sites. No gambling/porn themes.\n    -   **Includes:** Educational discussion of sensitive topics (drugs, gambling, adult, illegal), suggestive content (dating, lingerie) *without* explicit material, drugs for educational or news purposes (e.g., addiction recovery, medical cannabis research).\n\n-   **1

In [13]:
dataset_dict = dataset.train_test_split(test_size=0.004)

In [14]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['conversations', 'text'],
        num_rows: 33128
    })
    test: Dataset({
        features: ['conversations', 'text'],
        num_rows: 134
    })
})

In [15]:
# @title wandb init
import wandb
wandb.login()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [16]:
%env WANDB_WATCH=all
%env WANDB_SILENT=true

env: WANDB_WATCH=all
env: WANDB_SILENT=true


## Training

In [30]:
wandb.login(relogin=True)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/fishmon/.netrc


True

In [31]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
import os
from transformers.utils import logging
import wandb
from dotenv import load_dotenv

load_dotenv()  # Loads from .env file automatically

# 2. Verify token loading
if not os.getenv("WANDB_API_KEY"):
    raise ValueError("WANDB_API_KEY not found in .env file")

# 3. Initialize and upload
os.environ["WANDB_API_KEY"]=os.getenv("WANDB_API_KEY")

logging.set_verbosity_info()
project_name = "netpro-finetune" 
entity_name = "jordinia-netpro"
run_name = "llama-3.2-3b-unsloth-sft-2025-04-29"  # Set your desired run name

# Initialize WANDB (FIXED ENTITY/PROJECT)
try:
    run = wandb.init(
        entity=entity_name,
        project=project_name,
        name=run_name,
    )
    print("Successfully connected to WANDB!")
except Exception as e:
    print(f"Failed to initialize WANDB: {str(e)}")
    # Consider exiting if WANDB is critical
    # sys.exit(1)

Successfully connected to WANDB!


In [32]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['conversations', 'text'],
        num_rows: 33128
    })
    test: Dataset({
        features: ['conversations', 'text'],
        num_rows: 134
    })
})

In [35]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# Calculate training length (33k samples)
total_samples = 33000  # Your balanced dataset size
batch_size = 2 * 4  # batch_size * gradient_accum
steps_per_epoch = total_samples // batch_size  # ~4125 steps

args = TrainingArguments(
    output_dir               = "outputs",
    per_device_train_batch_size    = 2,
    gradient_accumulation_steps    = 4,
    warmup_ratio             = 0.1,           # ~10% of steps
    max_steps                = steps_per_epoch * 1,
    learning_rate            = 2e-5,
    fp16                     = not is_bfloat16_supported(),
    bf16                     = is_bfloat16_supported(),
    optim                    = "adamw_8bit",
    weight_decay             = 0.01,
    lr_scheduler_type        = "linear",      # linear decay
    eval_strategy      = "steps",
    eval_steps               = 500,           # every ~12% of the epoch
    save_strategy            = "steps",
    save_steps               = 500,
    save_total_limit         = 2,
    logging_steps            = 50,
    seed                     = 3407,
    report_to                = "wandb",
    load_best_model_at_end   = True,
    metric_for_best_model    = "eval_loss",
)

trainer = SFTTrainer(
    model               = model,
    tokenizer           = tokenizer,
    train_dataset       = dataset_dict["train"],
    eval_dataset        = dataset_dict["test"],
    dataset_text_field  = "text",
    max_seq_length      = max_seq_length,     # ← your pre-configured variable
    data_collator       = DataCollatorForSeq2Seq(
                             tokenizer = tokenizer,
                             padding = True,
                             pad_to_multiple_of = 8,
                             max_length          = max_seq_length,
                         ),
    dataset_num_proc    = 2,
    packing             = False,
    args                = args,
)

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     data_collator = DataCollatorForSeq2Seq(
#         tokenizer = tokenizer,
#         padding = True,                # ← Critical change
#         pad_to_multiple_of = 8,        # ← Aligns with GPU memory
#         max_length = max_seq_length,
#     ),
#     dataset_num_proc = 2,
#     packing = False, # Can make training 5x faster for short sequences.
#     args = TrainingArguments(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         # num_train_epochs = 1, # Set this for 1 full training run.
#         max_steps = 60,
#         learning_rate = 2e-4,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
#         report_to = "wandb", # Use this for WandB etc
#     ),
# )

PyTorch: setting up devices
PyTorch: setting up devices
Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 134/134 [00:01<00:00, 118.11 examples/s]
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


### Train on Completions Only

In [25]:
def train_on_responses_only_custom(trainer):
    """
    Custom version for website classification that:
    1. Preserves system prompt context
    2. Only trains on assistant responses (JSON + reasoning)
    3. Handles length consistency for batching
    """
    tokenizer = trainer.tokenizer
    
    # Manually define token sequences for your template
    SYSTEM_TOKENS = tokenizer.encode(
        "<|start_header_id|>system<|end_header_id|>\n\n", 
        add_special_tokens=False
    )
    USER_TOKENS = tokenizer.encode(
        "<|start_header_id|>user<|end_header_id|>\n\n", 
        add_special_tokens=False
    )
    ASSISTANT_TOKENS = tokenizer.encode(
        "<|start_header_id|>assistant<|end_header_id|>\n\n", 
        add_special_tokens=False
    )

    def custom_masking(examples):
        input_ids = examples["input_ids"]
        
        # Create labels if they don't exist
        labels = examples.get("labels", [ids.copy() for ids in input_ids])
        new_labels = []
        
        for seq_id, seq_labels in zip(input_ids, labels):
            n = len(seq_id)
            mask = [-100] * n
            i = 0
            
            while i < n:
                # Check for system prompt
                if seq_id[i:i+len(SYSTEM_TOKENS)] == SYSTEM_TOKENS:
                    i += len(SYSTEM_TOKENS)
                    continue
                    
                # Check for user prompt
                if seq_id[i:i+len(USER_TOKENS)] == USER_TOKENS:
                    i += len(USER_TOKENS)
                    continue
                    
                # Check for assistant prompt
                if seq_id[i:i+len(ASSISTANT_TOKENS)] == ASSISTANT_TOKENS:
                    start = i
                    i += len(ASSISTANT_TOKENS)
                    
                    # Find end of assistant response
                    while i < n:
                        if seq_id[i] == tokenizer.eos_token_id:
                            end = i
                            break
                        i += 1
                    else:
                        end = n
                    
                    # Unmask assistant response
                    mask[start:end] = seq_labels[start:end]
                    break
                    
                i += 1
            
            # Enforce length matching (critical fix)
            if len(mask) != len(seq_id):
                mask = mask[:len(seq_id)] + [-100] * (len(seq_id) - len(mask))
            
            new_labels.append(mask)
        
        return {"labels": new_labels}

    # Apply to datasets with length verification
    def apply_masking(dataset):
        dataset = dataset.map(
            custom_masking,
            batched=True,
            batch_size=1000,
            num_proc=4,
        )
        # Verify lengths
        for i in range(min(3, len(dataset))):
            assert len(dataset[i]["input_ids"]) == len(dataset[i]["labels"]), \
                f"Length mismatch in sample {i}"
        return dataset
    
    trainer.train_dataset = apply_masking(trainer.train_dataset)
    
    if trainer.eval_dataset is not None:
        trainer.eval_dataset = apply_masking(trainer.eval_dataset)
        
    return trainer

In [26]:
trainer = train_on_responses_only_custom(trainer)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Map (num_proc=4): 100%|██████████| 776/776 [00:01<00:00, 408.20 examples/s]


In [27]:
# Final verification (optional)
print("\n=== Final Pre-Train Check ===")
sample = trainer.train_dataset[0]
print("First sample labels preview:")
print("Input IDs length:", len(sample["input_ids"])) 
print("Labels length:", len(sample["labels"]))
print("Last 10 labels:", sample["labels"][-10:])  # Should show unmasked assistant tokens


=== Final Pre-Train Check ===
First sample labels preview:
Input IDs length: 6714
Labels length: 6714
Last 10 labels: [220, 330, 83029, 794, 220, 5313, 198, 534, 74694, 128009]


In [28]:
# Get a sample from the training set
sample_index = 5  # Try different indices
sample = trainer.train_dataset[sample_index]

# Decode the full input context
print("==== Full Input Context ====")
print(tokenizer.decode(sample["input_ids"]))
print("\n")

# Decode the labels with masking visualization
print("==== Training Targets (Masked) ====")
masked_labels = []
for token_id, label_id in zip(sample["input_ids"], sample["labels"]):
    if label_id == -100:
        # Show masked tokens as blank spaces
        masked_labels.append(" ")
    else:
        # Show actual token
        masked_labels.append(tokenizer.decode([token_id]))

print("".join(masked_labels))

==== Full Input Context ====
<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an Expert Website Classifier tasked with categorizing websites (using provided Domain & Content) into distinct categories: **0 - Benign**, **1 - Gambling**, **2 - Pornography**, or **3 - Harmful**. Output strict JSON including classification, reason, and a point-based confidence score (0-100). This aids a sophisticated website prediction system for digital safety.

### **Categories & Definitions:**

-   **0 - Benign:** General info, news, entertainment, services, e-commerce, educational sites, blogs, informational pages, and general entertainment sites. No gambling/porn themes.
    -   **Includes:** Educational discussion of sensitive topics (drugs, gambling, adult, illegal), suggestive content (dating, lingerie) *without* explicit material, drugs for educational or news purposes (e.g., addiction recovery, med

In [29]:
batch = trainer.train_dataset.select(range(2)).with_format("torch")[0]
print(batch["input_ids"])
print(batch["labels"])


tensor([128000, 128000, 128006,  ...,    534,  74694, 128009])
tensor([  -100,   -100,   -100,  ...,    534,  74694, 128009])


In [30]:
trainer.train_dataset

Dataset({
    features: ['conversations', 'text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 776
})

### Memory Stats

In [36]:
trainer.train_dataset

Dataset({
    features: ['conversations', 'text', 'input_ids', 'attention_mask'],
    num_rows: 33128
})

In [43]:
dataset_dict['test'][1]

{'conversations': [{'content': 'You are an Expert Website Classifier tasked with categorizing websites (using provided Domain & Content) into distinct categories: **0 - Benign**, **1 - Gambling**, **2 - Pornography**, or **3 - Harmful**. Output strict JSON including classification, reason, and a point-based confidence score (0-100). This aids a sophisticated website prediction system for digital safety.\n\n### **Categories & Definitions:**\n\n-   **0 - Benign:** General info, news, entertainment, services, e-commerce, educational sites, blogs, informational pages, and general entertainment sites. No gambling/porn themes.\n    -   **Includes:** Educational discussion of sensitive topics (drugs, gambling, adult, illegal), suggestive content (dating, lingerie) *without* explicit material, drugs for educational or news purposes (e.g., addiction recovery, medical cannabis research).\n\n-   **1 - Gambling:** Promotes/facilitates betting, casino, poker, lottery, wagering. Includes online bett

In [37]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090 Ti. Max memory = 23.551 GB.
3.441 GB of memory reserved.


In [38]:
trainer_stats = trainer.train()

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, conversations. If text, conversations are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 33,128 | Num Epochs = 1 | Total steps = 4,125
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 48,627,712/3,000,000,000 (1.62% trained)
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, conversations. If text, conversations are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 134
  Batch size = 2
Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.12 GiB. GPU 0 has a total capacity of 23.55 GiB of which 1.60 GiB is free. Including non-PyTorch memory, this process has 21.37 GiB memory in use. Of the allocated memory 20.84 GiB is allocated by PyTorch, and 191.72 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [44]:
# Run this before the automatic evaluation at step 500
trainer.evaluate(eval_dataset=trainer.eval_dataset)

The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, conversations. If text, conversations are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 134
  Batch size = 2


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.12 GiB. GPU 0 has a total capacity of 23.55 GiB of which 656.56 MiB is free. Including non-PyTorch memory, this process has 22.32 GiB memory in use. Of the allocated memory 21.76 GiB is allocated by PyTorch, and 220.72 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [45]:
print(f"Using bf16: {training_args.bf16}")

NameError: name 'training_args' is not defined

In [39]:
run.finish()

train/epoch,▁▂▃▃▄▅▆▆▇█
train/global_step,▁▂▃▃▄▅▆▆▇█
train/grad_norm,▃▂▁▂▁█▂▁▁▁
train/learning_rate,▁▂▃▄▅▆▇███
train/loss,█▅▁▁▁▁▁▁▁▁
train/epoch,0.12074
train/global_step,500
train/grad_norm,0.08471
train/learning_rate,2e-05
train/loss,0.0209


In [40]:
# Run evaluation immediately
eval_results = trainer.evaluate()
print(f"Validation loss at step 250: {eval_results['eval_loss']}")

The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text, conversations. If text, conversations are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 134
  Batch size = 2


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.12 GiB. GPU 0 has a total capacity of 23.55 GiB of which 1.13 GiB is free. Including non-PyTorch memory, this process has 21.83 GiB memory in use. Of the allocated memory 21.30 GiB is allocated by PyTorch, and 196.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [27]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1099.2794 seconds used for training.
18.32 minutes used for training.
Peak reserved memory = 5.771 GB.
Peak reserved memory for training = 2.33 GB.
Peak reserved memory % of max memory = 24.504 %.
Peak reserved memory for training % of max memory = 9.893 %.


In [36]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Load the system prompt from the file
with open('./prompt/labelling_promptv4.txt', 'r') as system_file:
    system_prompt = system_file.read()

# Load the label from the file
with open('./prompt/class_3_sample1.txt', 'r') as label_file:
    label = label_file.read()

# Define the messages
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"Classify the given URL as 0 (benign), 1 (gambling), 2 (pornography), or 3 (harmful). Output MUST be JSON.\n{label}"}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 800,
                   use_cache = True, temperature = 0.4, min_p = 0.5)

Classify the given URL as 0 (benign), 1 (gambling), 2 (pornography), or 3 (harmful). Output MUST be JSON.
\nDomain: jitutogel.net, Content: \"Whatsapp Telegram Livechat DAFTAR LOGIN BERANDA SLOT GAMES LIVE CASINO POKER ONLINE E-SPORTS ARCADE LOTTERY PROMOTION Jackpot Gaming Playstar TTG Slots Spadegaming RedTiger GMW CQ9 Gaming Live Gaming Evolution Gaming Sexy Gaming SAgaming HOgaming Gameplay OpusGaming Sports Gaming CMD368 SBOBET UBOBET TFGaming Ultraplay SabaEsports JituTogel: Serunya Game Online dengan Hadiah Uang Nyata! Penyedia Games Metode Pembayaran Bank Cimb Niaga BCA Danamon Permata Mandiri BNI BRI Panin Pulsa XL Axiata Tri Telkomsel Axis E-Money QRIS Dana OVO LinkAja Gopay ©2024 jitutogel. All rights reserved | 18+ LIVECHAT\""urls"urlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurlsurl

In [32]:
print(messages)

[{'role': 'system', 'content': 'You are an Expert Website Classifier tasked with categorizing websites (using provided Domain & Content) into distinct categories: **0 - Benign**, **1 - Gambling**, **2 - Pornography**, or **3 - Harmful**. Output strict JSON including classification, reason, and a point-based confidence score (0-100). This aids a sophisticated website prediction system for digital safety.\n\n### **Categories & Definitions:**\n\n-   **0 - Benign:** General info, news, entertainment, services, e-commerce, educational sites, blogs, informational pages, and general entertainment sites. No gambling/porn themes.\n    -   **Includes:** Educational discussion of sensitive topics (drugs, gambling, adult, illegal), suggestive content (dating, lingerie) *without* explicit material, drugs for educational or news purposes (e.g., addiction recovery, medical cannabis research).\n\n-   **1 - Gambling:** Promotes/facilitates betting, casino, poker, lottery, wagering. Includes online bett